# Libraries and Utility function

In [1]:
import time

start = time.time()

import networkit as nk
import numpy as np
import csv
from sklearn import preprocessing as pre

# working with text
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from nltk import sent_tokenize, word_tokenize
import scipy as sc

end = time.time()
print('Importing libraries and setting up parameters takes %.4f s' % (end-start))

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


Importing libraries and setting up parameters takes 10.2657 s


In [2]:
def build_graph(nodes, edges):
    g = nk.Graph(len(nodes)) # adding nodes

    for edge in edges:
        if not g.hasEdge(edge[0], edge[1]): # avoid multiple edges
            g.addEdge(edge[0], edge[1])
            
    return g

In [3]:
# a function to preprocess text
def preprocess(text, dg_removal=True, sw_removal=True, stemming=True):
    '''
    Preprocess text: stopword removal, stemming, digit removal
    
    Parameters
    ----------
    text: text on which preprocessing is applied
    dg_removal: whether to apply digit removal or not
    sw_removal: whether to apply stopword removal or not
    stemming: whether to apply stemming or not
    
    Returns
    -------
    the text after preprocessing
    '''
    result = text
    
    sw = set(nltk.corpus.stopwords.words('english')) # set of stopwords
    stemmer = nltk.stem.PorterStemmer() # stemmer
    
    if dg_removal:
        result = re.sub('[0-9]', '', result)
    
    if sw_removal:
        result = ' '.join([token for token in result.split() if token not in sw])
        
    if stemming:
        result = ' '.join([stemmer.stem(token) for token in result.split()])
    
    return result

# Reading data

In [4]:
path_data = '../data/' # path to the data
path_submission = '../submission/' # path to submission files

In [5]:
start = time.time()

# ====== read in node informations ====== #
with open(path_data + 'node_information.csv', 'r') as f:
    reader = csv.reader(f)
    node_info = list(reader)

end = time.time()
print('Reading node information takes %.4f s' % (end-start))

Reading node information takes 0.3039 s


In [6]:
start = time.time()

# ====== read training data as str ====== #
training = np.genfromtxt(path_data + 'training_set.txt', dtype=str)

end = time.time()
print('Reading training set takes %.4f s' % (end-start))

Reading training set takes 2.7073 s


In [7]:
start = time.time()

# ====== read testing data as str ====== #
testing = np.genfromtxt(path_data + 'testing_set.txt', dtype=str)

end = time.time()
print('Reading testing set takes %.4f s' % (end-start))

Reading testing set takes 0.1402 s


# Building the citation graph

In [8]:
start = time.time()

# ====== build the graph ====== #

nodes = [element[0] for element in node_info] # create index list to be passed as nodes
edges = [(nodes.index(element[0]), nodes.index(element[1])) for element in training if element[2] == '1']
g = build_graph(nodes, edges)

end = time.time()
print('Building the citation graph takes %.4f s' % (end-start))

Building the citation graph takes 177.7908 s


In [9]:
# check for general information of the graph
print('Number of vertices: %d' % g.numberOfNodes())
print('Number of edges (after multiple edges removal): %d' % g.numberOfEdges())

Number of vertices: 27770
Number of edges (after multiple edges removal): 334690


# Computing features

The list of features is described as follows, and the computation rule is the same for both training and testing set.

| Feature                | Explanation                                                        | Type      | Range   |
|:----------------------:|:------------------------------------------------------------------:|:---------:|:-------:|
| Temporal difference    | Difference in publication year (absolute value)                    | numerical | $\ge$ 0 |
| Common authors         | The number of common authors between two articles                  | numerical | $\ge$ 0 |
| Same journal           | Whether two articles are published in the same journal             | binary    | 0, 1    |
| Cosine similarity      | Cosine similarity between word vectors of abstracts                | numerical | [0,1]   |
| Title overlap          | Number of overlapping words in title                               | numerical | $\ge$ 0 |
| Degree difference      | Difference in measure of degrees of two nodes (absolute value)     | numerical | $\ge$ 0 |
| Common neighbors       | Number of common neighbors                                         | numercial | $\ge$ 0 |
| Jaccard coefficient    | Link-based Jaccard coefficient                                     | numerical | [0,1]   |
| Same cluster           | Check whether two nodes are in the same cluster                    | binary    | [0,1]   |
| PageRank difference    | Difference in PageRank index of two nodes (absolute value)         | numerical | $\ge$ 0 |
| Betweenness centrality | Difference in betweenness centrality of two nodes (absolute value) | numerical | $\ge$ 0 |
| In the same k-core     | Whether both nodes/one of them/none of them are in the same k-core | ordinal   |[0,0.5,1]|

In [10]:
# compute the dictionary of (ID-index) to accelerate access to node'ID
ID = dict(zip(nodes, [nodes.index(n) for n in nodes]))

## 1. Temporal difference

In [12]:
def temporal_difference(ds):
    '''
    Compute feature: Difference in publication year
    
    Parameters
    ----------
    ds: the dataset to compute
    
    Returns
    -------
    A numpy array where each entry corresponds to the temporal difference of a pair of nodes
    '''
    size = len(ds)
    temp_diff = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        src_info, dest_info = node_info[ID[src]], node_info[ID[dest]] # get the associated node information by index
        
        # compute the difference in publication year in absolute value (because we don't know which one cites the other)
        temp_diff[i] = abs(
            int(src_info[1]) - int(dest_info[1])
        )
        
    return temp_diff

In [13]:
start = time.time()

# compute the temporal difference
train_temp_diff = temporal_difference(training)

end = time.time()
print('Computing temporal difference for training set takes %.4f s' %(end-start))

Computing temporal difference for training set takes 1.6704 s


In [14]:
start = time.time()

# compute the temporal difference
test_temp_diff = temporal_difference(testing)

end = time.time()
print('Computing temporal difference for testing set takes %.4f s' %(end-start))

Computing temporal difference for testing set takes 0.1156 s


In [15]:
print('Training:', train_temp_diff[0:10])
print('Testing:', test_temp_diff[0:10])

Training: [0. 1. 2. 4. 5. 0. 4. 7. 0. 8.]
Testing: [0. 1. 2. 0. 5. 4. 0. 1. 7. 0.]


## 2. Number of common authors

In [16]:
def common_authors(ds):
    '''
    Compute feature: number of common authors
    
    Parameters
    ----------
    ds: dataset to compute feature from
    
    Returns
    -------
    A numpy array
    '''
    size = len(ds)
    common_auth = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        src_info, dest_info = node_info[ID[src]], node_info[ID[dest]] # get the associated node information by index
        
        # compute the difference in publication year in absolute value (because we don't know which one cites the other)
        common_auth[i] = len(
            set(src_info[3].split(','))
            .intersection(set(dest_info[3].split(',')))
        )
        
        
    return common_auth

In [17]:
start = time.time()

# compute the temporal difference
train_common_auth = common_authors(training)

end = time.time()
print('Computing the number of common authors for training set takes %.4f s' %(end-start))

Computing the number of common authors for training set takes 2.5344 s


In [18]:
start = time.time()

# compute the temporal difference
test_common_auth = common_authors(testing)

end = time.time()
print('Computing the number of common authors for testing set takes %.4f s' %(end-start))

Computing the number of common authors for testing set takes 0.1583 s


In [19]:
print('Training:', train_common_auth[0:10])
print('Testing:', test_common_auth[0:10])

Training: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
Testing: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


## 3. Same journal

In [20]:
def same_journal(ds):
    '''
    Compute feature: whether two articles are published in the same journal
    
    Parameters
    ----------
    ds: dataset to compute feature from
    
    Returns
    -------
    A numpy array of binary values (0|1)
    '''
    size = len(ds)
    same_journal = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        src_info, dest_info = node_info[ID[src]], node_info[ID[dest]] # get the associated node information by index
        
        # 1 if two articles are published in the same journal, 0 otherwise
        same_journal[i] = int(
            len(src_info[4])>0 and  # journal info of source not null
            len(dest_info[4])>0 and # journal info of dest not null
            src_info[4] == dest_info[4] # the same journal title
        )
        
        
    return same_journal

In [21]:
start = time.time()

# compute the temporal difference
train_same_journal = same_journal(training)

end = time.time()
print('Computing whether two articles are published in the same journal for training set takes %.4f s' %(end-start))

Computing whether two articles are published in the same journal for training set takes 1.4511 s


In [22]:
start = time.time()

# compute the temporal difference
test_same_journal = same_journal(testing)

end = time.time()
print('Computing whether two articles are published in the same journal for testing set takes %.4f s' %(end-start))

Computing whether two articles are published in the same journal for testing set takes 0.1035 s


In [23]:
print('Training:', train_same_journal[0:10])
print('Testing:', test_same_journal[0:10])

Training: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Testing: [0. 0. 1. 1. 0. 0. 0. 1. 0. 1.]


## 4. Cosine similarity in title + abstract

### 4.1 Cosine similarity with TD-IDF

In [12]:
start = time.time()

# ====== corpus is the set of titles + abstracts, apply preprocessing to each article ======#

# build the corpus
#nltk.download('stopwords') # uncomment if haven't downloaded stopwords
corpus = [preprocess(element[2] + ' ' + element[5], dg_removal=True, sw_removal=True, stemming=True) 
          for element in node_info]

end = time.time()
print('Building the corpus with preprocessing on words takes %.4f s' % (end-start))

Building the corpus with preprocessing on words takes 41.8818 s


In [41]:
start = time.time()

vectorizer = TfidfVectorizer(stop_words='english') # create a TF-IDF vectorizer
tfidf = vectorizer.fit_transform(corpus) # TD-IDF matrix of the entire corpus (set of abstracts)
print('TF-IDF matrix of shape:', tfidf.shape)

end = time.time()
print('Building TF-IDF matrix takes %.4f s' % (end-start))

TF-IDF matrix of shape: (27770, 17080)
Building TF-IDF matrix takes 1.4731 s


In [49]:
def cosine_sim_text(ds, vectors, is_w2v):
    '''
    Compute feature: cosine similarity in title and abstract, using normal cosine similarity on TF-IDF
    
    Parameters
    ----------
    ds: dataset to compute feature from
    
    Returns
    -------
    A numpy array of cosine values
    '''
    size = len(ds)
    cosines = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        
        # collect the cosine similarity
        src_vect, dest_vect = vectors[ID[src]], vectors[ID[dest]] # get the corresponding vector in TD-IDF matrix

        # compute cosine similarity
        cos = 1 - sc.spatial.distance.cosine(src_vect, dest_vect) if is_w2v else cosine_similarity(src_vect, dest_vect)
        
        cosines[i] = cos
        
    return cosines

In [26]:
start = time.time()

# compute the cosine similarity
train_cosine = cosine_sim_text(training, tfidf, False)

end = time.time()
print('Computing cosine similarity for training set takes %.4f s' %(end-start))

Computing cosine similarity for training set takes 372.6914 s


In [27]:
start = time.time()

# compute the cosine similarity
test_cosine = cosine_sim_text(testing, tfidf, False)

end = time.time()
print('Computing cosine similarity for testing set takes %.4f s' %(end-start))

Computing cosine similarity for testing set takes 19.3335 s


In [28]:
print('Training:', train_cosine[0:10])
print('Testing:', test_cosine[0:10])

Training: [0.19996622 0.06436945 0.02053711 0.05937844 0.09852643 0.39581923
 0.18722569 0.08627054 0.04181436 0.06044751]
Testing: [0.11804009 0.30786265 0.20753805 0.16112407 0.31824453 0.03466872
 0.02490266 0.19991048 0.         0.3283665 ]


### 4.2 Cosine similarity with word2vec

In [17]:
start = time.time()

# reading google vector
google_vecs = KeyedVectors.load_word2vec_format(path_data + 'GoogleNews-vectors-negative300.bin.gz', binary=True)

end = time.time()
print('Loading word2vec of google takes %.4f s' % (end-start))

INFO:gensim.models.utils_any2vec:loading projection weights from ../data/GoogleNews-vectors-negative300.bin.gz
INFO:gensim.models.utils_any2vec:loaded (3000000, 300) matrix from ../data/GoogleNews-vectors-negative300.bin.gz


Loading word2vec of google takes 141.5187 s


In [19]:
start = time.time()

# building documents for word2vec training
train_tag = []
total = len(corpus)
processed = 0
i = 0
#nltk.download('punkt') # uncomment if package 'punkt' not already downloaded
for x in corpus:
    words = []
    sentences = sent_tokenize(x)
    for s in sentences:
        words.extend(word_tokenize(s)) 
    doc = words
    i = i+1
    train_tag.append(doc)

end = time.time()
print('Building documents for word2vec training takes %.4f s' % (end-start))

[nltk_data] Downloading package punkt to /home/huong/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Building documents for word2vec training takes 15.1954 s


In [46]:
def word2vec(x_data,vector):
    print ("Loading GoogleNews-vectors-negative300.bin")
    google_vecs = vector
    print ("GoogleNews-vectors-negative300.bin loaded")
    
    print ("Averaging Word Embeddings...")
    x_data_embeddings = []
    total = len(x_data)
    for tagged_plot in x_data:
        count = 0  
        doc_vector = np.zeros(300)
        for sentence in tagged_plot:
            try:
                doc_vector += google_vecs[sentence]
            except KeyError:
                doc_vector += [0.0]*300
                continue

        x_data_embeddings.append(doc_vector)
            
    return np.array(x_data_embeddings)

In [47]:
start = time.time()

# get word embessings
x_embeddings = word2vec(train_tag, google_vecs)
print('Word embeddings of shape:', x_embeddings.shape)

end = time.time()
print('Embedding words takes %.4f s' % (end-start))

Loading GoogleNews-vectors-negative300.bin
GoogleNews-vectors-negative300.bin loaded
Averaging Word Embeddings...
Word embeddings of shape: (27770, 300)
Embedding words takes 12.8759 s


In [50]:
start = time.time()

# compute cosine similarity with word embeddings
train_cosine_sim_w2v = cosine_sim_text(training, x_embeddings, True)

end = time.time()
print('Computing cosine similarity using word embedding for training features takes %.4f s' % (end-start))

/home/huong/.local/lib/python3.5/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Computing cosine similarity using word embedding for training features takes 20.3391 s


In [51]:
start = time.time()

# compute cosine similarity with word embeddings
test_cosine_sim_w2v = cosine_sim_text(testing, x_embeddings, True)

end = time.time()
print('Computing cosine similarity using word embedding for testing features takes %.4f s' % (end-start))

/home/huong/.local/lib/python3.5/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Computing cosine similarity using word embedding for testing features takes 1.2045 s


In [52]:
print('Training:', train_cosine_sim_w2v[0:10])
print('Testing:', test_cosine_sim_w2v[0:10])

Training: [0.76839412 0.64235938 0.76174991 0.73752149 0.7232033  0.79878754
 0.79923392 0.68025815 0.4998241  0.59656393]
Testing: [0.72750916 0.69251008 0.61316547 0.60472135 0.37674628 0.6143832
 0.43079299 0.70911597 0.40506944 0.65525613]


## 5. Number of overlapped words in title

In [29]:
def overlap_title(ds):
    '''
    Compute feature: number of overlapping words in the title
    
    Parameters
    ----------
    ds: dataset to compute feature from
    
    Returns
    -------
    A numpy array of numerical values
    '''
    size = len(ds)
    overlap_title = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        src_info, dest_info = node_info[ID[src]], node_info[ID[dest]] # get the associated node information by index
        
        # collect the number of overlapping words in title
        src_title, dest_title = preprocess(src_info[2]).split(), preprocess(dest_info[2]).split()
        overlap_title[i] = len(
            set(src_title)
            .intersection(set(dest_title))
        )
        
    return overlap_title

In [30]:
start = time.time()

# compute the number of overlapping words in title
train_overlap_title = overlap_title(training)

end = time.time()
print('Computing number of overlapping words in title for training set takes %.4f s' %(end-start))

Computing number of overlapping words in title for training set takes 616.3972 s


In [31]:
start = time.time()

# compute the number of overlapping words in title
test_overlap_title = overlap_title(testing)

end = time.time()
print('Computing number of overlapping words in title for testing set takes %.4f s' %(end-start))

Computing number of overlapping words in title for testing set takes 32.3066 s


In [32]:
print('Training:', train_overlap_title[0:10])
print('Testing:', test_overlap_title[0:10])

Training: [2. 1. 0. 0. 0. 0. 0. 1. 0. 0.]
Testing: [0. 2. 1. 1. 0. 0. 1. 1. 0. 1.]


## 6. Maximum degree of a pair of nodes

In [89]:
def max_degrees(ds, g):
    '''
    Compute feature: <Maximum degrees among 2 nodes
    
    Parameters
    ----------
    ds: dataset to compute feature from
    g: the graph
    
    Returns
    -------
    A numpy array of numerical values
    '''
    size = len(ds)
    avg_degree = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        
        # collect the number of overlapping words in title
        src_deg = g.degree(ID[src])
        dest_deg = g.degree(ID[dest])
        avg_degree[i] = max(src_deg, dest_deg)
        #avg_degree[i] = float(src_deg + dest_deg)/2.0
        
    return avg_degree

In [90]:
start = time.time()

# compute the average degree
train_degrees = average_degrees(training, g)

end = time.time()
print('Computing the by-pair maximum degree for training set takes %.4f s' %(end-start))

Computing the by-pair maximum degree for training set takes 2.4667 s


In [91]:
start = time.time()

# compute the average degree
test_degrees = average_degrees(testing, g)

end = time.time()
print('Computing the by-pair maximum degree for testing set takes %.4f s' %(end-start))

Computing the by-pair maximum degree for testing set takes 0.1312 s


In [92]:
print('Training:', train_degrees[0:10])
print('Testing:', test_degrees[0:10])

Training: [ 12. 147.   5.  20.  24.  38. 739.  86. 100.  30.]
Testing: [ 59. 302. 739.  65. 150.  35.   4.  42.   7.  13.]


## 7. Number of common neighbors

In [37]:
def common_neighbors(ds, g):
    '''
    Compute feature: The number of common neighbors
    
    Parameters
    ----------
    ds: dataset to compute feature from
    g: the graph
    
    Returns
    -------
    A numpy array of numerical values
    '''
    size = len(ds)
    common_neigh = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        
        # collect the number of overlapping words in title
        common_neigh[i] = len(
            set(g.neighbors(ID[src]))
            .intersection(set(g.neighbors(ID[dest])))
        )
        
    return common_neigh

In [38]:
start = time.time()

# compute the average degree
train_common_neigh = common_neighbors(training, g)

end = time.time()
print('Computing the number of common neighbors for training set takes %.4f s' %(end-start))

Computing the number of common neighbors for training set takes 8.2263 s


In [39]:
start = time.time()

# compute the average degree
test_common_neigh = common_neighbors(testing, g)

end = time.time()
print('Computing the number of common neighbors for testing set takes %.4f s' %(end-start))

Computing the number of common neighbors for testing set takes 0.4318 s


In [40]:
print('Training:', train_common_neigh[0:10])
print('Testing:', test_common_neigh[0:10])

Training: [ 1. 20.  0.  0.  0. 14. 12.  0.  5.  0.]
Testing: [ 0. 24. 59. 21.  0.  0.  0.  6.  0.  4.]


## 8. Link-based Jaccard coefficient

In [41]:
def jaccard_coeff(ds, g):
    '''
    Compute feature: Link-based Jaccard coefficient
    
    Parameters
    ----------
    ds: dataset to compute feature from
    g: the graph
    
    Returns
    -------
    A numpy array of numerical values
    '''
    size = len(ds)
    coeff = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        
        # collect the number of overlapping words in title
        inters = len(
            set(g.neighbors(ID[src]))
            .intersection(set(g.neighbors(ID[dest])))
        ) # intersection of neighbors
        
        union = len(
            set(g.neighbors(ID[src]))
            .union(set(g.neighbors(ID[dest])))
        ) # union of neighbors
        
        coeff[i] = (float(inters)/float(union) if union != 0 else 0)
        
    return coeff

In [42]:
start = time.time()

# compute the average degree
train_jaccard_coeff = jaccard_coeff(training, g)

end = time.time()
print('Computing link-based Jaccard coefficient for training set takes %.4f s' %(end-start))

Computing link-based Jaccard coefficient for training set takes 19.7526 s


In [43]:
start = time.time()

# compute the average degree
test_jaccard_coeff = jaccard_coeff(testing, g)

end = time.time()
print('Computing link-based Jaccard coefficient for testing set takes %.4f s' %(end-start))

Computing link-based Jaccard coefficient for testing set takes 1.0697 s


In [44]:
print('Training:', train_jaccard_coeff[0:10])
print('Testing:', test_jaccard_coeff[0:10])

Training: [0.05882353 0.09708738 0.         0.         0.         0.23728814
 0.01522843 0.         0.03676471 0.        ]
Testing: [0.         0.07430341 0.06533776 0.22105263 0.         0.
 0.         0.10526316 0.         0.19047619]


## 9. Maximum by pair of PageRank index

In [108]:
# ====== compute PageRank index ====== #
start = time.time()

page_rank_g = nk.centrality.PageRank(g)
page_rank_g.run()

end = time.time()
print('Computing the PageRank index of the graph takes %.4f s' % (end-start))

Computing the PageRank index of the graph takes 0.2039 s


In [146]:
def max_pagerank(ds, pr):
    '''
    Compute feature: average of pagerank
    
    Parameters
    ----------
    ds: dataset to compute feature from
    g: the graph
    
    Returns
    -------
    A numpy array of numerical values
    '''
    size = len(ds)
    max_pr = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        
        # collect the average of betweenness centrality of 2 nodes
        # log to "dampen" too small values
        max_pr[i] = np.log(max(pr[ID[src]], pr[ID[dest]]))
        
    return max_pr

In [161]:
start = time.time()

# compute the average pagerank
train_pagerank = max_pagerank(training, page_rank_g.scores())

end = time.time()
print('Computing the by-pair maximum page rank for training set takes %.4f s' %(end-start))

Computing the by-pair maximum page rank for training set takes 1.7328 s


In [162]:
start = time.time()

# compute the average pagerank
test_pagerank = max_pagerank(testing, page_rank_g.scores())

end = time.time()
print('Computing the by-pair maximum page rank for testing set takes %.4f s' %(end-start))

Computing the by-pair maximum page rank for testing set takes 0.1327 s


In [163]:
print('Training:', train_pagerank[0:10])
print('Testing:', test_pagerank[0:10])

Training: [-10.4952174   -9.04507102 -11.03082025 -10.67156463 -10.4743869
 -10.27890873  -7.28060674  -9.25626085  -9.19986647  -9.55846355]
Testing: [ -9.71123221  -8.05014677  -7.28060674  -9.78108665  -8.84688464
 -10.35156937 -11.13700203  -9.81535402 -10.94645343 -10.8153264 ]


## 10. Maximum of betweenness centrality

In [169]:
# ====== compute betweenness centrality ====== #
start = time.time()

# use the traditional approach of betweeness computation
btwn = nk.centrality.Betweenness(g)
btwn.run()

end = time.time()
print('Compute betweenness centrality of every node in the graph takes %.4f s' % (end-start))

Compute betweenness centrality of every node in the graph takes 403.0792 s


In [184]:
def max_betweeness(ds, btwn):
    '''
    Compute feature: average in betweenness centrality
    
    Parameters
    ----------
    ds: dataset to compute feature from
    g: the graph
    
    Returns
    -------
    A numpy array of numerical values
    '''
    size = len(ds)
    max_btw = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        
        # collect the average of betweenness centrality of 2 nodes
        _max = max(btwn[ID[src]], btwn[ID[dest]])
        max_btw[i] = np.log(_max) if _max != 0 else 0.0
        
        #_avg = float(btwn[ID[src]] + btwn[ID[dest]])
        #avg_btw[i] = np.log(_avg/2.0) if _avg != 0.0 else 0.0
        
    return max_btw

In [185]:
start = time.time()

# compute the average degree
train_btwn = max_betweeness(training, btwn.scores())

end = time.time()
print('Computing the average betweenness for training set takes %.4f s' %(end-start))

Computing the average betweenness for training set takes 4.0639 s


In [186]:
start = time.time()

# compute the average degree
test_btwn = max_betweeness(testing, btwn.scores())

end = time.time()
print('Computing the average betweenness for testing set takes %.4f s' %(end-start))

Computing the average betweenness for testing set takes 0.2102 s


In [188]:
print('Training:', train_btwn[0:10])
print('Testing:', test_btwn[0:10])

Training: [10.5093708  11.39325368  9.53921731  8.04019582  9.38724288  8.87265376
 15.9955066  12.63640921 13.02476499 11.99064799]
Testing: [12.58866497 14.77807021 15.9955066  11.20495121 13.16245853 10.26706831
  5.53651846 11.07954426  7.44026286  8.02905778]


## 11. Core Decomposition

**Intuition:** retrieve the core of a network, where many articles are connected to each other. Given a pair of articles, if both are found in the core, they are likely to connect to each other (assign value 1). If one is in the core and one is not, they might be connect to each other (assign value 0.5). Otherwise, they are highly unlikely to connect to each other (value 0)

In [227]:
start = time.time()

core_decomp = nk.community.CoreDecomposition(g, storeNodeOrder=True)
core_decomp.run()
cover_g = core_decomp.getCover()
order = 15

end = time.time()
print('Core decomposition of the graph takes %.4f s' % (end-start))

Core decomposition of the graph takes 0.1228 s


In [228]:
# idx = 1
# for ss in cover_g.subsetSizes():
#     print('Subset of order %d has %d elements' % (idx, ss))
#     idx += 1

In [229]:
print('There are %d nodes that belong in %d-core decomposition of this graph' 
      % (len(cover_g.getMembers(order)), order))

There are 9647 nodes that belong in 15-core decomposition of this graph


In [230]:
def in_kcore(ds, kcore):
    '''
    Compute feature: whether a pair of nodes is found in the same k-core graph
    
    Parameters
    ----------
    ds: dataset
    kcore: the k-core graph after decomposition as a set of nodes index (ranged from 0 to 27,770)
    
    Returns
    -------
    A numpy array of ordinal values: 
        - 0 if both nodes are not in the kcores, 
        - 0.5 if one of them is in the kcores, 
        - 1 of they are both in the k-core
    '''
    size = len(ds)
    same_kcore = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        
        # compute whether two nodes are in the given kcore | one of them is in the kcore | none of them
        index_src = ID[src] # index of src
        index_dest = ID[dest] # index of dest
        
        if index_src in kcore and index_dest in kcore:
            result = 1.0
        elif index_src not in kcore and index_dest not in kcore:
            result = 0.0
        else:
            result = 0.5
            
        same_kcore[i] = result
        
    return same_kcore

In [231]:
start = time.time()

# compute the position of two nodes wrt k-core
train_in_kcore = in_kcore(training, cover_g.getMembers(order))

end = time.time()
print('Computing the in k-core feature for training set takes %.4f s' %(end-start))

Computing the in k-core feature for training set takes 1.0302 s


In [232]:
start = time.time()

# compute the position of two nodes wrt k-core
test_in_kcore = in_kcore(testing, cover_g.getMembers(order))

end = time.time()
print('Computing the in k-core feature for testing set takes %.4f s' %(end-start))

Computing the in k-core feature for testing set takes 0.1119 s


In [233]:
print('Training:', train_in_kcore[0:10])
print('Testing:', test_in_kcore[0:10])

Training: [0.  1.  0.  0.5 0.5 1.  1.  0.5 1.  0. ]
Testing: [1.  1.  1.  1.  0.5 0.5 0.  1.  0.  0. ]


## 12. Adamic-Adar coefficient

In [62]:
def adamic_adar_coeff(ds, g):
    '''
    Compute Adamic-Adar coefficient
    '''
    
    size = len(ds)
    aa_coeff = np.zeros(size)
    aa_index = nk.linkprediction.AdamicAdarIndex(g)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        
        aa_coeff[i] = aa_index.run(ID[src], ID[dest])
        
    return aa_coeff

In [63]:
start = time.time()

# compute the adamic-adar coefficient
train_aa_coeff = adamic_adar_coeff(training, g)

end = time.time()
print('Computing Adamic-Adar coefficient feature for training set takes %.4f s' %(end-start))

Computing Adamic-Adar coefficient feature for training set takes 4.8188 s


In [64]:
start = time.time()

# compute the adamic-adar coefficient
test_aa_coeff = adamic_adar_coeff(testing, g)

end = time.time()
print('Computing Adamic-Adar coefficient feature for testing set takes %.4f s' %(end-start))

Computing Adamic-Adar coefficient feature for testing set takes 0.2673 s


In [65]:
print('Training:', train_aa_coeff[0:10])
print('Testing:', test_aa_coeff[0:10])

Training: [0.51389834 4.32036615 0.         0.         0.         3.17502987
 2.46874101 0.         0.9428623  0.        ]
Testing: [ 0.          5.37797275 15.05361173  4.89942438  0.          0.
  0.          1.46868886  0.          1.27898053]


## 13. Katz index (read from already computed file)

In [ ]:
start = time.time()

# ====== read in katz index of training set ====== #
with open(path_data + 'katz_training.txt', 'r') as f:
    reader = csv.reader(f)
    train_katz_index = list(reader)

end = time.time()
print('Reading katz feature training takes %.4f s' % (end-start))

In [ ]:
start = time.time()

# ====== read in katz index of training set ====== #
with open(path_data + 'katz_testing.txt', 'r') as f:
    reader = csv.reader(f)
    test_katz_index = list(reader)

end = time.time()
print('Reading katz feature testing takes %.4f s' % (end-start))

## 13. Katz index (with ``centrality`` package)

In [213]:
start = time.time()

katz = nk.centrality.KatzCentrality(g)
katz.run()

end = time.time()
print('Computing katz centrality (with centrality package) takes %.4f s' % (end-start))

Computing katz centrality (with centrality package) takes 0.0929 s


In [220]:
def max_katz(ds, katz_scores):
    '''
    Compute Katz index between a pair of nodes (using centrality package and compute the average)
    '''
    size = len(ds)
    katz_result = np.zeros(size)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1]
        _katz = max(katz_scores[ID[src]], katz_scores[ID[dest]])
        katz_result[i] = np.log(_katz) if _katz != 0.0 else 0.0
        #katz_result[i] = float(katz_scores[ID[src]] + katz_scores[ID[dest]])/2.0
        
    return katz_result

In [221]:
start = time.time()

# compute the katz index
train_katz = max_katz(training, katz.scores())

end = time.time()
print('Computing Katz max feature for training set takes %.4f s' %(end-start))

Computing Katz max feature for training set takes 4.1459 s


In [222]:
start = time.time()

# compute the katz index
test_katz = max_katz(testing, katz.scores())

end = time.time()
print('Computing Katz average feature for testing set takes %.4f s' %(end-start))

Computing Katz average feature for testing set takes 0.2795 s


In [223]:
print('Training:', train_katz[0:10])
print('Testing:', test_katz[0:10])

Training: [-5.5804487  -4.2164855  -5.64652461 -5.25171866 -5.36014155 -4.96955029
 -2.88068326 -4.86388004 -4.29294528 -5.46850804]
Testing: [-4.69918791 -3.80632847 -2.88068326 -4.78584888 -4.22347698 -4.97580013
 -5.65542043 -5.30505257 -5.6155517  -5.50044342]


## 13. Katz index (with ``linkprediction`` package)

In [13]:
def katz_linkpred(ds, g):
    '''
    Compute Katz index between a pair of nodes (using linkprediction)
    '''
    size = len(ds)
    katz_result = np.zeros(size)
    katz = nk.linkprediction.KatzIndex(g)
    
    for i in range(size):
        src, dest = ds[i][0], ds[i][1] # get the source and dest ID
        katz_result[i] = katz.run(ID[src], ID[dest])
        if (i+1)%1000 == 0:
            print('Processed %d data' % (i+1))
        
    return katz_result

**Attention:** only run the two following cells when you really want to do it, because the computation takes around **9 hours** to complete.

In [21]:
start = time.time()

# set number of threads
nk.engineering.setNumberOfThreads(4)

# compute the katz index
train_katz_linkpred = katz_linkpred(training, g)

# end = time.time()
print('Computing Katz index feature for training set takes %.4f s' %(end-start))

Processed 1000 data
Processed 2000 data
Processed 3000 data
Processed 4000 data
Processed 5000 data
Processed 6000 data
Processed 7000 data
Processed 8000 data
Processed 9000 data
Processed 10000 data
Processed 11000 data
Processed 12000 data
Processed 13000 data
Processed 14000 data
Processed 15000 data
Processed 16000 data
Processed 17000 data
Processed 18000 data
Processed 19000 data
Processed 20000 data
Processed 21000 data
Processed 22000 data
Processed 23000 data
Processed 24000 data
Processed 25000 data
Processed 26000 data
Processed 27000 data
Processed 28000 data
Processed 29000 data
Processed 30000 data
Processed 31000 data
Processed 32000 data
Processed 33000 data
Processed 34000 data
Processed 35000 data
Processed 36000 data
Processed 37000 data
Processed 38000 data
Processed 39000 data
Processed 40000 data
Processed 41000 data
Processed 42000 data
Processed 43000 data
Processed 44000 data
Processed 45000 data
Processed 46000 data
Processed 47000 data
Processed 48000 data
P

Processed 379000 data
Processed 380000 data
Processed 381000 data
Processed 382000 data
Processed 383000 data
Processed 384000 data
Processed 385000 data
Processed 386000 data
Processed 387000 data
Processed 388000 data
Processed 389000 data
Processed 390000 data
Processed 391000 data
Processed 392000 data
Processed 393000 data
Processed 394000 data
Processed 395000 data
Processed 396000 data
Processed 397000 data
Processed 398000 data
Processed 399000 data
Processed 400000 data
Processed 401000 data
Processed 402000 data
Processed 403000 data
Processed 404000 data
Processed 405000 data
Processed 406000 data
Processed 407000 data
Processed 408000 data
Processed 409000 data
Processed 410000 data
Processed 411000 data
Processed 412000 data
Processed 413000 data
Processed 414000 data
Processed 415000 data
Processed 416000 data
Processed 417000 data
Processed 418000 data
Processed 419000 data
Processed 420000 data
Processed 421000 data
Processed 422000 data
Processed 423000 data
Processed 

In [86]:
start = time.time()

# compute the katz index
test_katz_linkpred = katz_index(testing, g)

end = time.time()
print('Computing Katz index feature for testing set takes %.4f s' %(end-start))

Processed 999 data
Processed 1999 data
Processed 2999 data
Processed 3999 data
Processed 4999 data
Processed 5999 data
Processed 6999 data
Processed 7999 data
Processed 8999 data
Processed 9999 data
Processed 10999 data
Processed 11999 data
Processed 12999 data
Processed 13999 data
Processed 14999 data
Processed 15999 data
Processed 16999 data
Processed 17999 data
Processed 18999 data
Processed 19999 data
Processed 20999 data
Processed 21999 data
Processed 22999 data
Processed 23999 data
Processed 24999 data
Processed 25999 data
Processed 26999 data
Processed 27999 data
Processed 28999 data
Processed 29999 data
Processed 30999 data
Processed 31999 data
Computing Katz index feature for testing set takes 1168.3391 s


In [104]:
print('Training:', train_katz_index[0:10])
print('Testing:', test_katz_index[0:10])

Training: [5.02650754e-03 5.51846733e-03 0.00000000e+00 1.38756250e-06
 1.37575000e-07 5.35477387e-03 5.39283918e-03 1.52531250e-08
 5.13756281e-03 6.25000000e-12]
Testing: [2.84559375e-07 6.32439694e-04 1.54608918e-03 5.32540828e-04
 1.70984375e-06 1.46984375e-07 0.00000000e+00 1.55151381e-04
 6.40625000e-10 1.01133166e-04]


In [24]:
with open(path_data + 'katz_training.txt', 'wb') as f:
    np.savetxt(f, train_katz_linkpred)

# Saving features

In [36]:
# list of selected features
features = [
    'temporal_difference', # 0
    'common_authors', # 1
    'same_journal', # 2
    'cosine_sim', # 3
    'overlapping_title', # 4
    'max_degrees', # 5
    'common_neighbors', # 6
    'jaccard_coefficient', # 7
    'max_pagerank', # 8
    'max_betweenness', # 9
    'in_kcore', # 10
    'adamic_adar', # 11
    'katz_index', # 12
    'cosine_sim_w2v', # 13
    'katz_linkpred' # 14
]

In [100]:
# ====== create array of training feature ====== #
training_features = np.array([
    train_temp_diff,
    train_common_auth,
    train_same_journal,
    train_cosine,
    train_overlap_title,
    train_avg_degrees,
    train_common_neigh,
    train_jaccard_coeff,
    train_pagerank,
    train_btwn,
    train_in_kcore,
    train_aa_coeff,
    train_katz,
    train_cosine_sim_w2v,
    train_katz_linkpred
]).T

In [42]:
# ====== Saving features (training_features) ====== #
with open(path_data + 'training_features.csv', 'w', newline='') as f:
    csv_out = csv.writer(f)
    csv_out.writerow(features)
    for row in training_features:
        csv_out.writerow(row)

In [102]:
# ====== create array of testing features ====== #
testing_features = np.array([
    test_temp_diff,
    test_common_auth,
    test_same_journal,
    test_cosine,
    test_overlap_title,
    test_avg_degrees,
    test_common_neigh,
    test_jaccard_coeff,
    test_pagerank,
    test_btwn,
    test_in_kcore,
    test_aa_coeff,
    test_katz,
    test_cosine_sim_w2v,
    test_katz_linkpred
]).T

In [43]:
# ====== Saving features (testing_features) ====== #
with open(path_data + 'testing_features.csv', 'w', newline='') as f:
    csv_out = csv.writer(f)
    csv_out.writerow(features)
    for row in testing_features:
        csv_out.writerow(row)

# Replacing/Appending features without recomputing

In [25]:
start = time.time()

# ====== read training features ====== #
orig_training_features = np.genfromtxt(path_data + 'training_features.csv', delimiter=',', skip_header=1, dtype=float)

end = time.time()
print('Reading training features takes %.4f s' % (end-start))

Reading training features takes 10.9378 s


In [27]:
start = time.time()

# ====== read testing features as str ====== #
orig_testing_features = np.genfromtxt(path_data + 'testing_features.csv', delimiter=',', skip_header=1, dtype=float)

end = time.time()
print('Reading testing features takes %.4f s' % (end-start))

Reading testing features takes 0.4623 s


In [28]:
print('Training features:', orig_training_features.shape)
print('Testing features:', orig_testing_features.shape)

Training features: (615512, 14)
Testing features: (32648, 14)


## Append katz link prediction to training features

In [40]:
# recompute katz by adding log
_train_katz = [np.log(_katz) if _katz != 0.0 else 0.0 for _katz in train_katz_linkpred]
print(_train_katz[0:10])

# append katz linkpred to training features
training_features = np.append(orig_training_features, np.reshape(_train_katz, (-1,1)), axis=1)
print('After concatenation, training features has shape:', training_features.shape)

[-5.293029862567551, -5.199655114063047, 0.0, -13.487961947295274, -15.799096614000588, -5.229766804031945, -5.222683282397631, -17.998481436853037, -5.271176472957152, -25.79843965218024]
After concatenation, training features has shape: (615512, 15)


In [41]:
# read katz linkpred testing
test_katz_linkpred = np.genfromtxt(path_data + 'katz_testing.txt')

# add log
_test_katz = [np.log(_katz) if _katz != 0.0 else 0.0 for _katz in test_katz_linkpred]
print(_test_katz[0:10])

# append katz linkpred to testing features
testing_features = np.append(orig_testing_features, np.reshape(_test_katz, (-1,1)), axis=1)
print('After concatenation, testing features has shape:', testing_features.shape)

[-15.072323905681971, -7.365925687758105, -6.472026643304163, -7.537850990784671, -13.279108565893594, -15.732939548334132, 0.0, -8.771109264434683, -21.168576853601774, -9.199072438008713]
After concatenation, testing features has shape: (32648, 15)


## Change average_degree to maximum_degree

In [155]:
index_avg_deg = 5
training_features[:,index_avg_deg] = train_degrees
testing_features[:,index_avg_deg] = test_degrees
print('After replacement, training features has shape:', training_features.shape)
print('After replacement, testing features has shape:', testing_features.shape)

After replacement, training features has shape: (615512, 14)
After replacement, testing features has shape: (32648, 14)


## Change avg_pagerank to maximum_pagerank

In [165]:
index_pr = 8
training_features[:,index_pr] = train_pagerank
testing_features[:,index_pr] = test_pagerank
print('After replacement, training features has shape:', training_features.shape)
print('After replacement, testing features has shape:', testing_features.shape)

After replacement, training features has shape: (615512, 14)
After replacement, testing features has shape: (32648, 14)


## Change avg_betweenness to maximum_betweenness

In [189]:
index_btwn = 9
training_features[:,index_btwn] = train_btwn
testing_features[:,index_btwn] = test_btwn
print('After replacement, training features has shape:', training_features.shape)
print('After replacement, testing features has shape:', testing_features.shape)

After replacement, training features has shape: (615512, 14)
After replacement, testing features has shape: (32648, 14)


## Change in_kcore to order 15

In [234]:
index_kcore = 10
training_features[:,index_kcore] = train_in_kcore
testing_features[:,index_kcore] = test_in_kcore
print('After replacement, training features has shape:', training_features.shape)
print('After replacement, testing features has shape:', testing_features.shape)

After replacement, training features has shape: (615512, 14)
After replacement, testing features has shape: (32648, 14)


## Change Katz centrality from average to max

In [224]:
index_katz = 12
training_features[:,index_katz] = train_katz
testing_features[:,index_katz] = test_katz
print('After replacement, training features has shape:', training_features.shape)
print('After replacement, testing features has shape:', testing_features.shape)

After replacement, training features has shape: (615512, 14)
After replacement, testing features has shape: (32648, 14)
